# Imports

In [1]:
import math
import gzip
import json
import os
import glob


import pandas as pd
import screed
from tqdm import tqdm
# import seaborn as sns

# %matplotlib inline

# Subset protein fasta files for only BUSCO mammalia ids

## Perform subsetting of fasta files from uniprot IDs

In [4]:
def subset_by_uniprot_ids(fasta, uniprot_ids):
    records_subset = []
    with screed.open(fasta) as records:
        for record in records:
            name = record['name']
            record_id = name.split()[0]
            uniprot_id = record_id.split('|')[1]
            if uniprot_id in uniprot_ids:
                records_subset.append(record)
    return records_subset

            
def write_fasta(output_fasta, records):
    with open(output_fasta, 'w') as f:
        for record in records:
            f.write(">{name}\n{sequence}\n".format(**record))

        

In [16]:
orthodb_busco_mammalia = pd.read_csv(
    f'{busco_mammalia_folder}/busco_mammalia__orthodb__to__uniprot__with_species.csv'
)
print(orthodb_busco_mammalia.shape)
orthodb_busco_mammalia.head()

print((set(orthodb_busco_mammalia['species_name'])))

(64199, 6)
{'Capra hircus', 'Camelus bactrianus', 'Ornithorhynchus anatinus', 'Rhinolophus sinicus', 'Homo sapiens', 'Phascolarctos cinereus', 'Macaca mulatta', 'Aotus nancymaae', 'Oryctolagus cuniculus', 'Nannospalax galili', 'Ceratotherium simum simum', 'Tupaia chinensis', 'Erinaceus europaeus', 'Peromyscus maniculatus bairdii', 'Chinchilla lanigera', 'Lipotes vexillifer', 'Sorex araneus', 'Mus musculus'}


In [14]:

uniprot_fasta = "/home/olga/data_lg/czbiohub-reference/uniprot/releases/2019_11/manually_downloaded/uniprot__taxonomy_Mammalia_9MAMM_40674.fasta.gz"
output_folder = "/home/olga/data_lg/czbiohub-reference/uniprot/releases/2019_11/manually_downloaded/mammalia_busco_subsets/"

for (species_id, species_name), df in orthodb_busco_mammalia.groupby(["species", "species_name"]):
    species_uniprot_ids = set(df.external_db_gene_id)
    species_records = subset_by_uniprot_ids(uniprot_fasta, species_uniprot_ids)
    new_fasta = f"{output_folder}{species_id}__{species_name.lower().replace(' ', '_')}.fasta"
    write_fasta(new_fasta, species_records)
